In [ ]:
import numpy as np
import math

In [ ]:
def read_patient_data(filepath):
    data = {}
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            key, value = line.split("=")
            data[key.strip()] = value.strip()
    return data


In [ ]:
def winsorized_minmax(x, lower, upper):
    """
    Winsorized Min–Max scaling to [0,1]
    """
    x_w = max(min(x, upper), lower)
    return (x_w - lower) / (upper - lower)


def invert(x):
    """Инверсия шкалы (больше = лучше → больше = хуже)"""
    return 1.0 - x


In [ ]:
SYMPTOM_MAP = {
    "none": 0.0,
    "mild": 0.3,
    "moderate": 0.6,
    "severe": 1.0
}


In [ ]:
def compute_acute_risk(data, iob_max=6.0):
    # Приведение типов
    glucose = float(data["glucose"])
    trend = abs(float(data["trend"]))
    ketones = float(data["ketones"])
    iob = float(data["iob"])
    meal_time = float(data["time_since_meal"])
    symptoms = data["symptoms"].lower()
    hr = float(data["hr"])
    hr_base = float(data["hr_base"])

    # Нормировка
    r_glucose = winsorized_minmax(glucose, 3.9, 20.0)
    r_trend = winsorized_minmax(trend, 0.0, 0.2)
    r_ketones = winsorized_minmax(ketones, 0.0, 3.0)
    r_iob = winsorized_minmax(iob, 0.0, iob_max)

    # Время с еды: риск убывает до 4 часов
    meal_scaled = winsorized_minmax(meal_time, 0.0, 4.0)
    r_meal = invert(meal_scaled)

    r_symptoms = SYMPTOM_MAP.get(symptoms, 0.0)

    delta_hr = abs(hr - hr_base)
    r_hr = winsorized_minmax(delta_hr, 0.0, 40.0)

    acute_vector = [
        r_glucose,
        r_trend,
        r_ketones,
        r_iob,
        r_meal,
        r_symptoms,
        r_hr
    ]

    return np.array(acute_vector)


In [ ]:
def compute_chronic_risk(data):
    hba1c = float(data["hba1c"])
    tir = float(data["tir"])
    cv = float(data["cv"])
    hypo = float(data["severe_hypo"])
    acr = float(data["acr"])
    ldl = float(data["ldl"])
    duration = float(data["duration_years"])
    comorb = float(data["comorbidities"])

    r_hba1c = winsorized_minmax(hba1c, 5.0, 12.0)
    r_tir = invert(winsorized_minmax(tir, 0.0, 100.0))
    r_cv = winsorized_minmax(cv, 5.0, 45.0)
    r_hypo = winsorized_minmax(hypo, 0.0, 5.0)
    r_acr = winsorized_minmax(acr, 3.0, 300.0)
    r_ldl = winsorized_minmax(ldl, 1.8, 5.0)
    r_duration = winsorized_minmax(duration, 0.0, 30.0)
    r_comorb = winsorized_minmax(comorb, 0.0, 3.0)

    chronic_vector = [
        r_hba1c,
        r_tir,
        r_cv,
        r_hypo,
        r_acr,
        r_ldl,
        r_duration,
        r_comorb
    ]

    return np.array(chronic_vector)


In [ ]:
def polygon_area(radii):
    """
    Площадь многоугольника в полярных координатах
    """
    n = len(radii)
    angle = 2 * math.pi / n
    s = 0.0
    for i in range(n):
        r1 = radii[i]
        r2 = radii[(i + 1) % n]
        s += r1 * r2
    return 0.5 * math.sin(angle) * s


def normalized_risk(area, n):
    """
    Нормированная доля риска (0–1)
    """
    max_area = 0.5 * math.sin(2 * math.pi / n) * n
    return area / max_area


In [ ]:
# Загрузка данных
data = read_patient_data("patient_data.txt")

# Расчёт рисков
acute_vec = compute_acute_risk(data)
chronic_vec = compute_chronic_risk(data)

# Площади
acute_area = polygon_area(acute_vec)
chronic_area = polygon_area(chronic_vec)

# Нормированные коэффициенты риска
acute_risk_score = normalized_risk(acute_area, len(acute_vec))
chronic_risk_score = normalized_risk(chronic_area, len(chronic_vec))

# Вывод
print("Acute normalized vector:", np.round(acute_vec, 3))
print("Chronic normalized vector:", np.round(chronic_vec, 3))

print(f"\nAcute risk coefficient: {acute_risk_score:.3f}")
print(f"Chronic risk coefficient: {chronic_risk_score:.3f}")
